In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tenant_id/oauth2/token"}

dbutils.fs.mount(
source = "abfss://tokyo-2020-olympic-data@tokyo2020olympicdata.dfs.core.windows.net", # container@ storageacc name
mount_point = "/mnt/tokyo2020olympic",
extra_configs = configs)


Out[1]: True

In [0]:
%fs
ls "/mnt/tokyo2020olympic"

path,name,size,modificationTime
dbfs:/mnt/tokyo2020olympic/raw-data/,raw-data/,0,1693431555000
dbfs:/mnt/tokyo2020olympic/transformed-data/,transformed-data/,0,1693431560000


In [0]:
athletes = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyo2020olympic/raw-data/athletes.csv")
athletes.show()

+--------------------+--------------------+-------------------+
|                Name|                 NOC|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [0]:
coaches = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyo2020olympic/raw-data/coaches.csv")
entriesgender = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyo2020olympic/raw-data/entriesgender.csv")
medals = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyo2020olympic/raw-data/medals.csv")
teams = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/mnt/tokyo2020olympic/raw-data/teams.csv")


In [0]:
athletes.printSchema()
coaches.printSchema()
entriesgender.printSchema()
medals.printSchema()
teams.printSchema()

root
 |-- Name: string (nullable = true)
 |-- NOC: string (nullable = true)
 |-- Discipline: string (nullable = true)

root
 |-- Name: string (nullable = true)
 |-- NOC: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)

root
 |-- Rank: integer (nullable = true)
 |-- Team/NOC: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)

root
 |-- Name: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- NOC: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
#Top Gold Medals
top_gold = medals.orderBy("Gold", ascending = False).select("Team/NOC", "Gold").show()

+--------------------+----+
|            Team/NOC|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
|           Australia|  17|
|         Netherlands|  10|
|              France|  10|
|             Germany|  10|
|               Italy|  10|
|              Canada|   7|
|              Brazil|   7|
|         New Zealand|   7|
|                Cuba|   7|
|             Hungary|   6|
|   Republic of Korea|   6|
|              Poland|   4|
|      Czech Republic|   4|
|               Kenya|   4|
|              Norway|   4|
+--------------------+----+
only showing top 20 rows



In [0]:
#Percentage of gender distribution for each entry
percent_entry_by_gender = entriesgender.withColumn('Female_Percent', (entriesgender['Female']/entriesgender['Total'])*100).withColumn('Male_Percent', (entriesgender['Male']/entriesgender['Total'])*100)
percent_entry_by_gender.show()

+--------------------+------+----+-----+------------------+------------------+
|          Discipline|Female|Male|Total|    Female_Percent|      Male_Percent|
+--------------------+------+----+-----+------------------+------------------+
|      3x3 Basketball|    32|  32|   64|              50.0|              50.0|
|             Archery|    64|  64|  128|              50.0|              50.0|
| Artistic Gymnastics|    98|  98|  196|              50.0|              50.0|
|   Artistic Swimming|   105|   0|  105|             100.0|               0.0|
|           Athletics|   969|1072| 2041| 47.47672709456149| 52.52327290543851|
|           Badminton|    86|  87|  173| 49.71098265895954| 50.28901734104046|
|   Baseball/Softball|    90| 144|  234| 38.46153846153847| 61.53846153846154|
|          Basketball|   144| 144|  288|              50.0|              50.0|
|    Beach Volleyball|    48|  48|   96|              50.0|              50.0|
|              Boxing|   102| 187|  289|35.294117647

In [0]:
athletes.write.mode("overwrite").option("header", 'true').csv("/mnt/tokyo2020olympic/transformed-data/athletes")
coaches.write.option("header", 'true').csv("/mnt/tokyo2020olympic/transformed-data/coaches")
entriesgender.write.option("header", 'true').csv("/mnt/tokyo2020olympic/transformed-data/entriesgender")
medals.write.option("header", 'true').csv("/mnt/tokyo2020olympic/transformed-data/medals")
teams.write.option("header", 'true').csv("/mnt/tokyo2020olympic/transformed-data/teams")